<div style="text-align: center;">
    <h1>Codes Nomenclature des Activités Françaises (NAF) </h1>
</div>


Le but de ce notebook est de télécharger une liste de code NAF.

La NAF, nomenclature d'activités française, est une nomenclature des activités économiques productives, principalement élaborée pour faciliter l'organisation de l'information économique et sociale.

Le code NAF (ou APE) est construit de la manière suivante :

<p align="center">
  <img src="../img/codeape.png" alt="imagemanquante" width="30%" />
</p>



Plus d'informations sur le site de l'[INSEE](https://www.insee.fr/fr/information/2406147).

Il s'agit d'extraire de ces données les colonnes suivantes :

- codeNAF : on prendra la version rev2 de la nomenclature
- Intitule : intitulé du code NAF



In [1]:
! pip install -q xlrd

In [2]:
import os
import pandas as pd
import re

# 1. Téléchargement des données


Les codes Naf fournis par l'INSEE peuvent étre téléchargés sur [ce lien](https://www.insee.fr/fr/statistiques/fichier/2120875/int_courts_naf_rev_2.xls). 


In [12]:
# paramètres
url = "https://www.insee.fr/fr/statistiques/fichier/2120875/int_courts_naf_rev_2.xls"
usecols = ["Code", " Intitulés de la  NAF rév. 2, version finale "]

df = pd.read_excel(url, usecols=usecols)
df.columns = ["codeNaf", "libelleNaf"]

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2109 entries, 0 to 2108
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   codeNaf     1728 non-null   object
 1   libelleNaf  1728 non-null   object
dtypes: object(2)
memory usage: 33.1+ KB


# 2. Nettoyage des données

Dans la colonne 'Code', toutes les mailles sont présentes (Division / Classe et sous Classe).

On ne garde que la maille la plus fine de ce découpage, c'est-à-dire les codes de la forme "99.00Z".

In [15]:
# function to match the pattern
def match(string):
    return bool(re.match(r"\d{2}\.\d{2}[A-Z]", string))


df["codeNaf"] = df["codeNaf"].astype(str)

# apply the function to the column
df["match"] = df["codeNaf"].apply(match)

# filter the dataframe
df = df[df["match"] == True].drop("match", axis=1)

# 3. Sauvegarde du fichier

In [17]:
df.to_parquet("../Données nationales/NAF.parquet", index=False)